In [25]:
# Wikis

# https://en.wikipedia.org/wiki/Lists_of_victims_of_the_September_11_attacks

# https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(A%E2%80%93G)

# https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(H%E2%80%93N)

# https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(O%E2%80%93Z)

In [26]:
from bs4 import BeautifulSoup as bs
import requests
import csv
import pandas as pd
from pptx import Presentation
from pptx.util import Inches, Pt



urls = [
    "https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(A%E2%80%93G)",
    "https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(H%E2%80%93N)",
    "https://en.wikipedia.org/wiki/List_of_victims_of_the_September_11_attacks_(O%E2%80%93Z)"
]

data = []

for url in urls:
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')

    table = soup.find_all(class_="wikitable")

    if table:
        rows = table[0].find_all('tr')
        for row in rows[2:]:
            cells = row.find_all('td')
            if len(cells) >= 6:  # Check for at least 6 columns in the row
                name_cell = cells[0]
                age_cell = cells[1]
                job_cell = cells[6]  # Extracting data from the 6th column
                name = name_cell.get_text(strip=True)
                age = age_cell.get_text(strip=True)
                job = job_cell.get_text(strip=True)
                data.append([name, age, job])  # Include job data

# Create a Pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age', 'Job'])

df['Job'] = df['Job'].str.title()

presentation = Presentation()


# Define a function to set font size for text elements
def set_font_size(element, font_size):
    for run in element.runs:
        run.font.size = Pt(font_size)

# Iterate through the DataFrame and add a slide for each row
for index, row in df.iterrows():
    slide = presentation.slides.add_slide(presentation.slide_layouts[5])
    title = slide.shapes.title
    title.text = f"{row['Name']}"

    # Create a text box for displaying age and job
    left = Inches(1)
    top = Inches(2)
    width = Inches(8)
    height = Inches(1)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    tf = text_box.text_frame

    # Add text for age and job
    age_p = tf.add_paragraph()
    age_p.text = f"Age: {row['Age']}"
    job_p = tf.add_paragraph()
    job_p.text = f"Occupation: {row['Job']}"

    # Set font size for title and text elements
    set_font_size(age_p, 32)   # Set age font size to 18 points
    set_font_size(job_p, 32)   # Set job font size to 18 points

    # Enable text wrapping for age and job paragraphs (on the text box)
    text_box.text_frame.word_wrap = True  # Enable text wrapping

# Save the PowerPoint presentation
presentation.save('9/11.pptx')




# Write data to a CSV file

# with open('victims_updated.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(['Name', 'Age', 'Job'])  # Write header
#     csvwriter.writerows(data)            # Write data rows

# print("Data exported to victims_updated.csv")
